Dans ce code, différentes bibliothèques ont été utilisées pour traiter une BDD (OpenFoodFacts) sous forme de fichier CSV contenant des données sur les allergènes alimentaires. Ces bibliothèques ont permis de séparer les données en dictionnaires français et anglais, de normaliser les mots et de nettoyer les données, ainsi que d'utiliser des modèles de traitement du langage naturel (NLP) comme spaCy et NLTK pour détecter les allergènes dans des phrases spécifiques. 


In [19]:
import pandas as pd
import numpy as np
import spacy

In [1]:
try:
    # Charger uniquement la colonne "allergens" du fichier CSV
    df = pd.read_csv(r'C:\Users\boss\Desktop\STAGE\CODE\NLP\cleaned_openfoodfacts.csv', usecols=['allergens'], sep='\t', nrows=1417719, engine='python')
except pd.errors.ParserError as e:
    error_line = int(str(e).split(' ')[-1])
    df = pd.read_csv('openfoodfacts.csv', usecols=['allergens'], error_bad_lines=False, skiprows=range(1, error_line), sep='\t')
    print(f"Ligne {error_line}: {open('openfoodfacts.csv').readlines()[error_line-1]}")


In [5]:
# Extraire les données de la colonne "image_ingredients_url"
allergens = df['allergens']

In [10]:
# Supprimer les lignes vides et celles contenant des valeurs NaN
allergens = allergens.dropna(how='all').dropna()

In [11]:
allergens.head()

,allergens
11,en:mustard
31,"en:eggs,en:mustard"
48,"en:gluten,en:milk"
60,"en:fish,en:gluten,en:milk"
72,en:milk


In [12]:
# Sauvegarder le DataFrame résultant au format CSV
allergens.to_csv(r'C:\Users\boss\Desktop\STAGE\CODE\NLP\CSVfile\allergens.csv', index=False)

APRES AVOIR RECUPERER LE FICHIER CSV DES ALLERGENES, SEPARER CE FICHIER CSV EN DEUX DICTIONNAIRE L'UN POUR LE FRANCAIS ET L'UN POUR L'ANGLAIS SUIVANT LES TAGS

In [59]:
def process_csv(input_file):
    # Charger le fichier CSV
    df = pd.read_csv(input_file, header=None, names=['tags'])

    # Initialiser les dictionnaires pour les tags "en:" et "fr:"
    en_dict = {}
    fr_dict = {}

    # Commencer le parcours du DataFrame
    for index, row in df.iterrows():
        # Séparer les éléments par des virgules et supprimer les guillemets
        elements = [elem.strip('"') for elem in row['tags'].split(',')]

        # Parcourir chaque élément et séparer le tag du contenu
        for elem in elements:
            tag_and_content = elem.split(':')
            if len(tag_and_content) == 2:
                tag, content = tag_and_content
                if tag == 'en':
                    en_content = content.strip()
                    en_dict[en_content] = en_content
                elif tag == 'fr':
                    fr_content = content.strip()
                    fr_dict[fr_content] = fr_content

    return en_dict, fr_dict


In [60]:
#TEST AVEC LE DICTIONNAIRE
en_dict, fr_dict = process_csv(r"C:\Users\boss\Desktop\STAGE\CODE\NLP\CSVfile\allergens.csv")

print(en_dict)
print(fr_dict)

{'mustard': 'mustard', 'eggs': 'eggs', 'gluten': 'gluten', 'milk': 'milk', 'fish': 'fish', 'sesame-seeds': 'sesame-seeds', 'soybeans': 'soybeans', 'nuts': 'nuts', 'none': 'none', 'sulphur-dioxide-and-sulphites': 'sulphur-dioxide-and-sulphites', 'Barley Malt Extract': 'Barley Malt Extract', 'peanuts': 'peanuts', 'celery': 'celery', 'Skimmed': 'Skimmed', 'crustaceans': 'crustaceans', 'Lait': 'Lait', 'Shellfish': 'Shellfish', 'COCONUT': 'COCONUT', 'molluscs': 'molluscs', 'Wheat Gluten': 'Wheat Gluten', 'Phenylalanine': 'Phenylalanine', 'Pineapple': 'Pineapple', 'Pineapple Juice': 'Pineapple Juice', 'Flour': 'Flour', 'Cajou': 'Cajou', 'Huile de beurre': 'Huile de beurre', 'Cocoa butter': 'Cocoa butter', 'Vanilla': 'Vanilla', 'Whey Protein isolate': 'Whey Protein isolate', 'CONTAINS WHEAT': 'CONTAINS WHEAT', 'Pectin': 'Pectin', 'Corn': 'Corn', 'lupin': 'lupin', 'ALMOND BUTTER': 'ALMOND BUTTER', 'ALMOND FLOUR': 'ALMOND FLOUR', 'COCONUT OIL': 'COCONUT OIL', 'milkfat': 'milkfat', 'LEMON': 'LEM

In [61]:
len_en_dict = len(en_dict)
len_fr_dict = len(fr_dict)

print("Longueur du dictionnaire 'en:' :", len_en_dict)
print("Longueur du dictionnaire 'fr:' :", len_fr_dict)

Longueur du dictionnaire 'en:' : 355
Longueur du dictionnaire 'fr:' : 1195


NORMALISER LES MOTS ET LES RENDRE MINUSCULE ET EFFECTUER LE NETTOYAGE DES MOTS QUI NE SONT PAS EN FR ou EN ANGLAIS POUR LES DEUX DICTIONNAIRES

In [ ]:
# Normaliser et nettoyer le dictionnaire 'en_dict'
to_remove_en = []
for word in en_dict.copy():
    normalized_word = word.lower()  # Normalisation en minuscules
    if not normalized_word.isalpha():
        to_remove_en.append(word)
    else:
        en_dict[normalized_word] = en_dict.pop(word)
for word in to_remove_en:
    en_dict.pop(word)

# Normaliser et nettoyer le dictionnaire 'fr_dict'
to_remove_fr = []
for word in fr_dict.copy():
    normalized_word = word.lower()  # Normalisation en minuscules
    if not normalized_word.isalpha():
        to_remove_fr.append(word)
    else:
        fr_dict[normalized_word] = fr_dict.pop(word)
for word in to_remove_fr:
    fr_dict.pop(word)

In [62]:
# Obtenir la longueur des dictionnaires
len_en_dict = len(en_dict)
len_fr_dict = len(fr_dict)

print("Longueur du dictionnaire 'en:' :", len_en_dict)
print("Longueur du dictionnaire 'fr:' :", len_fr_dict)

# Afficher les dictionnaires résultants
print("Dictionnaire 'en:' :", en_dict)
print("Dictionnaire 'fr:' :", fr_dict)

Longueur du dictionnaire 'en:' : 157
Longueur du dictionnaire 'fr:' : 446
Dictionnaire 'en:' : {'mustard': 'mustard', 'eggs': 'eggs', 'gluten': 'gluten', 'milk': 'milk', 'fish': 'fish', 'soybeans': 'soybeans', 'nuts': 'nuts', 'none': 'none', 'peanuts': 'peanuts', 'celery': 'celery', 'skimmed': 'Skimmed', 'crustaceans': 'crustaceans', 'shellfish': 'Shellfish', 'molluscs': 'molluscs', 'phenylalanine': 'Phenylalanine', 'pineapple': 'Pineapple', 'flour': 'FLOUR', 'cajou': 'Cajou', 'vanilla': 'Vanilla', 'pectin': 'Pectin', 'lupin': 'lupin', 'milkfat': 'MILKFAT', 'lemon': 'LEMON', 'almondmilk': 'Almondmilk', 'cashewmilk': 'Cashewmilk', 'caffeine': 'CAFFEINE', 'sunflower': 'Sunflower', 'coconut': 'Coconut', 'yeast': 'yeast', 'coconuts': 'Coconuts', 'durum': 'Durum', 'honey': 'Honey', 'thiamine': 'THIAMINE', 'avoine': 'Avoine', 'lait': 'Lait', 'orge': 'orge', 'gelatin': 'gelatin', 'poisson': 'Poisson', 'chocolate': 'Chocolate', 'herring': 'Herring', 'metabisulphite': 'Metabisulphite', 'ceuf': 

TEST RANDOM

In [40]:
allergenes = {
    "céréales contenant du gluten": ["blé", "seigle", "orge", "avoine", "épeautre", "kamut", "triticale"],
    "crustacés": ["crevettes", "crabes", "homards", "langoustines", "huîtres", "moules", "palourdes", "escargots", "calmars", "pieuvres"],
    "œufs": ["œufs de poule", "œufs de caille", "œufs de canard", "œufs en poudre", "œufs liquides"],
    "poissons": ["thon", "saumon", "maquereau", "truite", "haddock", "poissons fumés", "sardines", "anchois"],
    "arachides": ["arachides"],
    "soja": ["soja", "tofu", "sauce soja", "lécithine de soja", "huile de soja", "farine de soja"],
    "lait et produits laitiers": ["lait de vache", "lait de chèvre", "lait de brebis", "lait en poudre", "crème", "fromages", "beurre", "yaourt", "glace", "poudre de lactosérum"],
    "fruits à coque": ["anacardes", "pacanes", "noix de macadamia", "noix du brésil", "amandes amères", "noix de pin", "noix de cajou", "pistaches", "noisettes", "noix de coco"],
    "sésame": ["graines de sésame", "huile de sésame", "tahini (purée de sésame)"],
    "moutarde": ["graines de moutarde", "poudre de moutarde", "sauce à la moutarde"],
    "céleri": ["racine de céleri", "feuilles de céleri", "graines de céleri"],
    "lupin": ["farine de lupin", "graines de lupin"],
    "mollusques": ["huîtres", "moules", "palourdes", "coques", "bigorneaux", "escargots", "calmars", "pieuvres"],
    "anhydride sulfureux et sulfites": ["sulfites de sodium", "sulfites de potassium", "sulfites de calcium", "vin", "produits séchés"],
}

In [41]:
def detect_allergen_in_sentence(sentence, allergen_dict):
    detected_allergens = []

    words = sentence.lower().split()  # Diviser la phrase en mots en minuscules

    for allergen, allergen_list in allergen_dict.items():
        for item in allergen_list:
            if item.lower() in words:
                detected_allergens.append(allergen)
            elif " ".join(words).find(item.lower()) != -1:
                detected_allergens.append(allergen)

    return detected_allergens



In [43]:
sentence = "J'adore les amandes, mais je suis allergique aux oeufs et aux sulfites de sodium."
detected_allergens = detect_allergen_in_sentence(sentence, allergenes)

if detected_allergens:
    print("Les allergènes détectés dans la phrase sont :", detected_allergens)
else:
    print("Aucun allergène n'a été détecté dans la phrase.")

Les allergènes détectés dans la phrase sont : ['anhydride sulfureux et sulfites']


 la correspondance de motifs : 

In [ ]:
# Charger le modèle de traitement du langage naturel de spaCy
nlp = spacy.load("fr_core_news_sm")

In [ ]:
def detect_allergen_in_sentence(sentence, allergen_dict):
    detected_allergens = []

    doc = nlp(sentence)

    for allergen, allergen_list in allergen_dict.items():
        for item in allergen_list:
            # Recherche de correspondance exacte de l'expression d'allergène
            if item.lower() in sentence.lower():
                detected_allergens.append(allergen)

            # Recherche de correspondance de motifs avec spaCy
            pattern = [{"LOWER": word.lower()} for word in item.split()]
            if any(pattern_matcher(doc, [pattern])):
                detected_allergens.append(allergen)

    return detected_allergens

def pattern_matcher(doc, patterns):
    matcher = spacy.matcher.Matcher(nlp.vocab)
    matcher.add("PatternMatcher", patterns)
    matches = matcher(doc)
    return matches

In [44]:
sentence = "J'adore les amandes, mais je suis allergique aux œufs et aux sulfites de sodium."
detected_allergens = detect_allergen_in_sentence(sentence, allergenes)

if detected_allergens:
    print("Les allergènes détectés dans la phrase sont :", detected_allergens)
else:
    print("Aucun allergène n'a été détecté dans la phrase.")


Les allergènes détectés dans la phrase sont : ['anhydride sulfureux et sulfites', 'anhydride sulfureux et sulfites']
